# Combining together the piecewise voltage simulation

In [73]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import xarray as xr
import pandas as pd
import scipy.io as sio
import sys
import os
import glob
import copy
import pathlib as pth
import importlib
import math
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.spice.splopter as spl
import flopter.spice.tdata as td
import flopter.core.ivdata as iv
import flopter.core.fitters as fts
import flopter.core.fitdata as fd
import flopter.core.lputils as lpu
import flopter.core.constants as c
import flopter.spice.inputparser as inp
import flopter.spice.normalise as nrm
import flopter.spice.utils as spu

In [74]:
importlib.reload(lpu)

<module 'flopter.core.lputils' from '/home/jleland/coding/projects/flopter/flopter/core/lputils.py'>

# Preamble of necessary stuff

This bit creates all teh necessary variables for plotting to commence, including loading the dataset, creating a denormaliser, extracting relevant probe dimensions from an input file and defining a few functions for ease of manipulation. 

In [75]:
spice_dir = pth.Path('/home/jleland/data/external_big/spice/')
os.chdir(spice_dir)

In [76]:
lps = lpu.MagnumProbes()

flush_probe = copy.deepcopy(lps.probe_l)
flush_probe.theta_p = 0.0
flush_probe.d_perp = 0.0

angled_probe = copy.deepcopy(lps.probe_l)
angled_probe.d_perp = 0.0
print(angled_probe.theta_p)

0.17453292519943295


In [77]:
skippable_scans = set(['marconi/spice2/piecewise/flat_flush_piecewise']) 
single_sims = set()

In [115]:
# scans_searchstr = '*/*/sheath_exp/*'
# scans_searchstr = '*/*/new_sheath_exp/*'
scans_searchstr = '*/*/piecewise/*'
# angles_search_str = '/*[!.{yml, inp}]/backup*'
angles_search_str = '/*[!.{yml, inp}]'

non_standard_variables = {'t', 'ProbePot', 'npartproc', 'Nz', 'Nzmax', 'Ny', 'count', 
                          'Npc', 'snumber', 'nproc', 'by', 'bz', 'rho01', 'rho02', 'rho'}
desired_variables = (td.DEFAULT_REDUCED_DATASET | non_standard_variables) - {td.OBJECTSCURRENTFLUXE, td.OBJECTSCURRENTFLUXI}

all_run_dirs = {}
scans = glob.glob(scans_searchstr)
scans = set(scans) - skippable_scans
for scan in scans:
    if scan in single_sims:
        all_run_dirs[scan] = [scan]
    else:
        all_run_dirs[scan] = glob.glob(scan + angles_search_str)



scans = list(scans)
scans.sort()

In [79]:
for i, scan in enumerate(scans):
    print(f"[{i}]: {scan}")
    for j, run in enumerate(all_run_dirs[scan]):
        print(f"\t[{i},{j}]: {'/'.join(run.split('/')[-2:])}")

[0]: marconi/spice2/piecewise/flat_flush_fixedpot
	[0,0]: flat_flush_fixedpot/alpha_yz_-12.0__pot_-12.0
	[0,1]: flat_flush_fixedpot/alpha_yz_-10.0__pot_-1.0
	[0,2]: flat_flush_fixedpot/alpha_yz_-10.0__pot_-10.0
	[0,3]: flat_flush_fixedpot/alpha_yz_-10.0__pot_-12.0
	[0,4]: flat_flush_fixedpot/alpha_yz_-10.0__pot_-2.0
	[0,5]: flat_flush_fixedpot/alpha_yz_-10.0__pot_-3.0
	[0,6]: flat_flush_fixedpot/alpha_yz_-10.0__pot_-4.0
	[0,7]: flat_flush_fixedpot/alpha_yz_-10.0__pot_-5.0
	[0,8]: flat_flush_fixedpot/alpha_yz_-10.0__pot_-6.0
	[0,9]: flat_flush_fixedpot/alpha_yz_-10.0__pot_-7.0
	[0,10]: flat_flush_fixedpot/alpha_yz_-10.0__pot_-8.0
	[0,11]: flat_flush_fixedpot/alpha_yz_-10.0__pot_-9.0
	[0,12]: flat_flush_fixedpot/alpha_yz_-10.0__pot_0.0
	[0,13]: flat_flush_fixedpot/alpha_yz_-12.0__pot_-1.0
	[0,14]: flat_flush_fixedpot/alpha_yz_-12.0__pot_-10.0
	[0,15]: flat_flush_fixedpot/alpha_yz_-12.0__pot_-2.0
	[0,16]: flat_flush_fixedpot/alpha_yz_-12.0__pot_-3.0
	[0,17]: flat_flush_fixedpot/alpha_yz_-

In [121]:
spl_path = spice_dir / all_run_dirs[scans[0]][14]
print(spl_path)

/home/jleland/data/external_big/spice/marconi/spice2/piecewise/flat_flush_fixedpot/alpha_yz_-12.0__pot_-10.0


## Start with the one

In [122]:
bups = list(spl_path.glob('backup*'))
bups.sort()
final_state_path = spl_path / bups[-1].name

In [123]:
splopter =  spl.Splopter(final_state_path, reduce=desired_variables, ignore_tzero_fl=True, version=2.14,
                         store_dataframe_fl=True, check_voltage_error_fl=False)

splopter.prepare(find_se_temp_fl=False, homogenise_fl=False)

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/piecewise/flat_flush_fixedpot/alpha_yz_-12.0__pot_-10.0/backup_20201117-0825) doesn't seem to be valid.
Continuing anyway.


In [124]:
splopter.tdata.t_dict.keys()

dict_keys(['__header__', '__version__', '__globals__', 'Nz', 'Nzmax', 'Ny', 'count', 'Npc', 'dt', 'dz', 'nproc', 'q', 'm', 'Temp', 'rho', 'Pot', 'Potvac', 't', 'snumber', 'by', 'bz', 'npartproc', 'ProbePot', 'objectsenum', 'objectscurrenti', 'objectscurrente', 'rho01', 'rho02', 'alphayz', 'alphaxz', 'mkspar3', 'sliceproc', 'impactdiag01', 'nodiagreg', 'emmitdiag02', 'histlimits', 'mksB', 'mksmainionq', 'bx', 'eHistSEx3', 'iHistTopx3', 'edges', 'ksi', 'fvperparraycount', 'solw02', 'diagm', 'orientation', 'Escz', 'equipotm', 'fvlimits', 'mkspar2', 'dPHIqn', 'flagm', 'eHistTopx2', 'itertime', 'floatconstant', 'mksn0', 'hpos', 'diaghistories', 'zg', 'objectspowerfluxi', 'flag', 'impactdiag02', 'solw01', 'irel', 'eHistSEx2', 'mu', 'tau', 'QnPot', 'surfacematrix', 'iHistSEx2', 'solns02', 'iHistTopx2', 'injrate', 'iHistSEx3', 'Escy', 'fvbin', 'iHistSEx1', 'timehistory', 'pchi', 'emmitdiag01', 'Nc', 'Esct', 'objectspowerfluxe', 'edgecharge', 'eHistTopx3', 'fvarrays', 'iHistTopx1', 'totalenergy

In [125]:
fig, ax = plt.subplots()

ax.plot(splopter.tdata.t_dict['ProbePot'])

for i in range(4):
    line_pos = i * len(splopter.tdata.t_dict['ProbePot']) / 4
    ax.axvline(x=line_pos, **c.AX_LINE_DEFAULTS)

In [126]:
# splopter.plot_2d_variable(b_arrow_loc=(100, 100))
fig, ax = plt.subplots()

# print(splopter.tdata.pot.shape)
# ax.plot(splopter.tdata.pot[:,200])
im = ax.pcolormesh(splopter.tdata.objectsenum)
fig.colorbar(im)

In [127]:

fig, ax = plt.subplots(2)

im = ax[0].pcolormesh(splopter.tdata.t_dict['rho01'])
fig.colorbar(im, ax=ax[0])
im = ax[1].pcolormesh(splopter.tdata.t_dict['rho02'])
fig.colorbar(im, ax=ax[1])

# x_slice = 200
# for x_slice in np.arange(128,429,10):
#     ax[0].axvline(x=x_slice, **c.AX_LINE_DEFAULTS)
#     ax[1].plot(splopter.tdata.rho[94:,x_slice])


In [14]:
probe_indices = [2,]

I_e = 0.0
I_i = 0.0

tdata = splopter.tdata
for index in probe_indices:
    I_e += np.squeeze(tdata.objectscurrente)[index]
    I_i += np.squeeze(tdata.objectscurrenti)[index]
V = np.squeeze(tdata.diagnostics[c.DIAG_PROBE_POT])

In [15]:
fig, ax = plt.subplots(2, sharex=True)
I_tot = (I_e + I_i)
ax[0].plot(I_tot[::100])

ax[1].plot(I_e[::100], label='e')
ax[1].plot(I_i[::100], label='i')
ax[1].legend()

In [16]:
half_len = int(len(I_tot) / 2)

In [17]:
fig, ax = plt.subplots(2, sharex=True)

ax[0].plot(I_tot[half_len:])

ax[1].plot(I_e[half_len:], label='e')
ax[1].plot(I_i[half_len:], label='i')
ax[1].legend()

/home/jleland/.pyenv/versions/3.6.9/lib/python3.6/tkinter/__init__.py:749: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args)
/home/jleland/.pyenv/versions/3.6.9/lib/python3.6/tkinter/__init__.py:1705: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  return self.func(*args)


In [18]:
theta = float(spl_path.name.split('_')[2])
pot = float(spl_path.name.split('_')[5])

In [19]:
I_tot_avg = np.mean(I_tot[half_len:])
I_tot_std = np.std(I_tot[half_len:]) / np.sqrt(half_len)

print(I_tot_avg, I_tot_std)

36.65248462443065 0.013942353408986703


In [20]:
# ds = xr.Dataset({'voltage': (['time'], iv['V'][:sweep_length]),
#                  'current': (['time'], iv['I'][:sweep_length]),
#                  'shot_time': (['time'], iv['t'][:sweep_length]),
#                  'start_time': iv['t'][0]},
#                 coords={'time': relative_t[:sweep_length], 'direction': 'up',
#                         'probe': probe_designations[i]})

# Combine it all together!

In [37]:
labels = ['theta', 'voltage', 
          'current', 'd_current', 'derr_current', 
          'current_e', 'd_current_e', 'derr_current_e', 
          'current_i', 'd_current_i', 'derr_current_i']
pw_df = pd.DataFrame(columns=labels)

In [38]:
probe_indices = [2,]

for i, scan in enumerate(scans):
    for j, run in enumerate(all_run_dirs[scan]):
        spl_path = spice_dir / run
        bups = list(spl_path.glob('backup*'))
        bups.sort()
        final_state_path = spl_path / bups[-1].name
        splopter =  spl.Splopter(final_state_path, reduce=desired_variables, ignore_tzero_fl=True, 
                                 check_voltage_error_fl=False)
        splopter.prepare(find_se_temp_fl=False, homogenise_fl=False)
        
        I_e = 0.0
        I_i = 0.0
        for index in probe_indices:
            I_e += np.squeeze(splopter.tdata.objectscurrente)[index]
            I_i += np.squeeze(splopter.tdata.objectscurrenti)[index]
        I_tot = I_e + I_i
        
        pw_df = pw_df.append({
            'theta': float(spl_path.name.split('_')[2]),
            'voltage': float(spl_path.name.split('_')[5]),
            'current': np.mean(I_tot[half_len:]),
            'd_current': np.std(I_tot[half_len:]),
            'derr_current': np.std(I_tot[half_len:]) / np.sqrt(half_len),
            'current_i': np.mean(I_i[half_len:]),
            'd_current_i': np.std(I_i[half_len:]),
            'derr_current_i': np.std(I_i[half_len:]) / np.sqrt(half_len),
            'current_e': np.mean(I_e[half_len:]),
            'd_current_e': np.std(I_e[half_len:]),
            'derr_current_e': np.std(I_e[half_len:]) / np.sqrt(half_len)
        }, ignore_index=True)
        
        
pw_df       

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/piecewise/flat_flush_fixedpot/alpha_yz_-12.0__pot_-12.0/backup_20201117-0831) doesn't seem to be valid.
Continuing anyway.
Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/piecewise/flat_flush_fixedpot/alpha_yz_-10.0__pot_-1.0/backup_20201117-0835) doesn't seem to be valid.
Continuing anyway.
Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/piecewise/flat_flush_fixedpot/alpha_yz_-10.0__pot_-10.0/backup_20201117-0708) doesn't seem to be valid.
Continuing anyway.
Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/piecewise/flat_flush_fixedpot/alpha_yz_-10.0__pot_-12.0/backup_20201117-0708) doesn't se

,theta,voltage,current,d_current,derr_current,current_e,d_current_e,derr_current_e,current_i,d_current_i,derr_current_i
0,-12.0,-12.0,35.342953,5.956226,0.013620,-0.010335,0.101817,0.000233,35.353288,5.955661,0.013618
1,-10.0,-1.0,-168.891039,15.995121,0.036575,-186.470465,15.854532,0.036254,17.579426,4.298539,0.009829
2,-10.0,-10.0,30.744934,5.780728,0.013218,-0.063494,0.252396,0.000577,30.808429,5.776388,0.013208
3,-10.0,-12.0,31.350957,5.797181,0.013256,-0.008532,0.092594,0.000212,31.359489,5.796408,0.013254
4,-10.0,-2.0,-121.246157,13.732644,0.031402,-142.260764,13.371590,0.030576,21.014607,4.720739,0.010795
5,-10.0,-3.0,-79.573143,13.650867,0.031215,-103.348080,12.575527,0.028756,23.774938,5.085640,0.011629
6,-10.0,-4.0,2.164415,8.137277,0.018607,-25.067921,6.185302,0.014144,27.232336,5.390991,0.012327
7,-10.0,-5.0,19.561762,6.431097,0.014706,-8.926346,3.464413,0.007922,28.488108,5.513585,0.012608
8,-10.0,-6.0,25.979538,5.914170,0.013524,-3.168723,1.964107,0.004491,29.148261,5.608992,0.012826
9,-10.0,-7.0,28.542205,5.739117,0.013123,-1.170515,1.122483,0.002567,29.712720,5.657999,0.012938


In [39]:
pw_df.loc[pw_df.theta == -8.0].set_index('voltage').to_xarray().drop('theta').expand_dims({'theta':[-8.0]})

<xarray.Dataset>
Dimensions:         (theta: 1, voltage: 17)
Coordinates:
  * theta           (theta) float64 -8.0
  * voltage         (voltage) float64 -1.0 -10.0 -12.0 ... -18.0 -20.0 -25.0
Data variables:
    current         (theta, voltage) float64 -135.5 25.94 26.36 ... 27.48 27.94
    d_current       (theta, voltage) float64 13.83 5.254 5.284 ... 5.374 5.411
    derr_current    (theta, voltage) float64 0.03163 0.01201 ... 0.01229 0.01237
    current_e       (theta, voltage) float64 -150.4 -0.05006 ... -7.893e-06
    d_current_e     (theta, voltage) float64 13.5 0.2234 ... 0.004292 0.00281
    derr_current_e  (theta, voltage) float64 0.03086 0.0005109 ... 6.425e-06
    current_i       (theta, voltage) float64 14.9 25.99 26.37 ... 27.48 27.94
    d_current_i     (theta, voltage) float64 3.962 5.251 5.283 ... 5.374 5.411
    derr_current_i  (theta, voltage) float64 0.009059 0.01201 ... 0.01237

In [40]:
combo_ds = xr.concat([pw_df.loc[pw_df.theta == theta].set_index('voltage').to_xarray().drop('theta').expand_dims({'theta':[theta]}) 
 for theta in [-8.0,-10.0,-12.0]], dim='theta')

In [41]:
combo_ds['theta_rads'] = np.radians(-combo_ds['theta'])
# combo_ds['theta_rads'] = -combo_ds['theta']
combo_ds = combo_ds.expand_dims({'probe': ['0']})
combo_ds['theta_p_rads'] = combo_ds['probe'] 
combo_ds['recession'] = combo_ds['probe'] 
combo_ds = combo_ds.sel(probe='0')

In [72]:
combo_ds.to_netcdf('piecewise_sheath_exp_ivs.nc')

In [43]:
swept_ds = xr.load_dataset('new_sheath_exp_ivs.nc')
swept_ds

<xarray.Dataset>
Dimensions:               (recession: 2, theta: 9, theta_p: 2, voltage: 401)
Coordinates:
  * recession             (recession) float64 0.0 0.001
    theta_p_desc          (theta_p) object 'flat' 'angled'
  * theta_p               (theta_p) float64 0.0 10.0
  * theta                 (theta) float64 3.0 4.0 5.0 6.0 ... 12.0 20.0 30.0
  * voltage               (voltage) float64 -9.95 -9.9 -9.85 ... 9.95 10.0 10.05
    recession_desc        (recession) object 'flush' 'recessed'
    theta_p_rads          (theta_p) float64 0.0 0.1745
    theta_rads            (theta) float64 0.05236 0.06981 ... 0.3491 0.5236
Data variables:
    str_iv_chi2           (recession, theta_p, theta) float64 6.743e+03 ... 159.0
    str_iv_reduced_chi2   (recession, theta_p, theta) float64 43.5 ... 1.026
    str_iv_I_sat          (recession, theta_p, theta) float64 -11.35 ... -71.91
    str_iv_d_I_sat        (recession, theta_p, theta) float64 3.13 ... 1.296
    str_iv_a              (recession, theta_p, theta) float64 1.868e-15 ... 0.04223
    str_iv_d_a            (recession, theta_p, theta) float64 0.12 ... 0.005092
    str_iv_T_e            (recession, theta_p, theta) float64 1.995 ... 0.9402
    str_iv_d_T_e          (recession, theta_p, theta) float64 0.3035 ... 0.02061
    str_iv_V_f            (recession, theta_p, theta) float64 -4.721 ... -2.776
    str_iv_d_V_f          (recession, theta_p, theta) float64 0.05557 ... 0.006002
    str_iv_run_dir        (recession, theta_p, theta) object 'marconi/spice2/new_sheath_exp/flat_flush/alpha_yz_-3.0' ... 'marconi/spice2/new_sheath_exp/angled_recessed/alpha_yz_-30.0'
    str_iv_voltage_min    (recession, theta_p, theta) float64 -9.95 ... -9.95
    str_iv_voltage_max    (recession, theta_p, theta) float64 -2.05 ... -2.05
    norm_iv_chi2          (recession, theta_p, theta) float64 138.3 ... 153.5
    norm_iv_reduced_chi2  (recession, theta_p, theta) float64 1.281 ... 1.073
    norm_iv_I_sat         (recession, theta_p, theta) float64 -6.476 ... -72.86
    norm_iv_d_I_sat       (recession, theta_p, theta) float64 0.1268 ... 0.7461
    norm_iv_a             (recession, theta_p, theta) float64 0.1512 ... 0.04027
    norm_iv_d_a           (recession, theta_p, theta) float64 0.01033 ... 0.003633
    norm_iv_voltage_min   (recession, theta_p, theta) float64 -5.454 ... -7.219
    norm_iv_voltage_max   (recession, theta_p, theta) float64 -0.004398 ... -0.01927
    ion_chi2              (recession, theta_p, theta) float64 127.9 ... 196.4
    ion_reduced_chi2      (recession, theta_p, theta) float64 1.184 ... 1.373
    ion_I_sat             (recession, theta_p, theta) float64 6.465 ... 79.16
    ion_d_I_sat           (recession, theta_p, theta) float64 0.04734 ... 0.4406
    ion_a                 (recession, theta_p, theta) float64 0.1515 ... 0.01446
    ion_d_a               (recession, theta_p, theta) float64 0.004415 ... 0.00207
    ion_voltage_min       (recession, theta_p, theta) float64 0.01708 ... 0.05173
    ion_voltage_max       (recession, theta_p, theta) float64 3.569 ... 4.404
    current               (recession, theta_p, theta, voltage) float64 -10.29 ... 915.7
    current_e             (recession, theta_p, theta, voltage) float64 -0.02646 ... -918.8
    current_i             (recession, theta_p, theta, voltage) float64 10.32 ... 3.105
    d_current             (recession, theta_p, theta, voltage) float64 0.2479 ... 6.614
    d_current_e           (recession, theta_p, theta, voltage) float64 0.01171 ... 6.695
    d_current_i           (recession, theta_p, theta, voltage) float64 0.2477 ... 0.495
    time                  (recession, theta_p, theta, voltage) float64 0.0 ... 2.581

## Start looking at data

In [45]:
fig, ax = plt.subplots()

fixed_plot_ds = combo_ds.sel(voltage=slice(-10.0,0))
sweep_plot_ds = swept_ds.sel(theta_p=0.0, recession=0.0, theta=slice(8,12), voltage=slice(-10,0))

fixed_plot_ds['current'].plot.line(x='voltage', hue='theta', ax=ax)
(-sweep_plot_ds['current']).plot.line(x='voltage', ax=ax)
ax.axhline(y=0, **c.AX_LINE_DEFAULTS)

In [48]:
fig, ax = plt.subplots(2)

se_fitter = fts.IonCurrentSEFitter()
thetas = []
seps = []
d_seps = []
for theta in combo_ds.theta.values:
    fixed_plot_ds = combo_ds.sel(theta=theta, voltage=slice(-25,-10))

    cl_voltage = np.float_power(np.abs(fixed_plot_ds.voltage.values + 4), 0.75)
    
#     ax[0].plot(fixed_plot_ds.pot.values, fixed_plot_ds.I_e.values)
    ax[0].plot(fixed_plot_ds.voltage.values, fixed_plot_ds.current_i.values)
    ax[1].plot(cl_voltage, fixed_plot_ds.current_i.values, label=theta)
    
    fit_data = se_fitter.fit(cl_voltage, fixed_plot_ds.current_i.values, sigma=fixed_plot_ds.derr_current_i.values)
    ax[1].plot(*fit_data.get_fit_plottables(), label='a={:.3g}'.format(fit_data.get_sheath_exp()))
    ax[1].legend()
    
    thetas.append(theta)
    seps.append(fit_data.get_sheath_exp())
    d_seps.append(fit_data.get_sheath_exp_err())
    

In [49]:
print(thetas, seps, 1/np.tan(-np.radians(thetas)))

[-8.0, -10.0, -12.0] [0.013246865664096224, 0.018304079543884168, 0.021236248601817248] [7.11536972 5.67128182 4.70463011]


In [50]:
combo_ds['ion_a'] = (combo_ds['theta'] * 0) + np.array(seps)

In [56]:
fig, ax = plt.subplots()

plot_decomp(combo_ds, sheath_label='ion_a', mode=1, ax=ax, colour='tab:blue')


(<matplotlib.axes._subplots.AxesSubplot at 0x7f478d89e518>,
 <flopter.core.fitdata.FitData2 at 0x7f478d862b00>)

## Vary the trimming to test sensitivity

In [57]:
cb_palette = ['#377eb8', '#ff7f00', '#4daf4a', 
              '#f781bf', '#a65628', '#984ea3', 
              '#999999', '#e41a1c', '#dede00',
              '#377eb8', '#ff7f00', '#4daf4a', 
              '#f781bf', '#a65628', '#984ea3', 
              '#999999', '#e41a1c', '#dede00']

In [59]:
trim_values = np.flip(combo_ds.voltage.values[2:-5])
trim_values

array([ -5.,  -6.,  -7.,  -8.,  -9., -10., -12., -14., -16., -18.])

In [61]:
fig, ax = plt.subplots(2, figsize=[8,10])

# combo_ds.sel(pot=slice(-25,-4))['I_tot'].plot.line(x='pot', hue='theta', ax=ax[0])

for i, lower_trim in enumerate(trim_values):
    se_fitter = fts.IonCurrentSEFitter()
    thetas = []
    seps = []
    d_seps = []
    ax[0].axvline(x=np.float_power(np.abs(lower_trim + 4), 3/4), **c.AX_LINE_DEFAULTS)
    for theta in combo_ds.theta.values:
        fixed_plot_ds = combo_ds.sel(theta=theta, voltage=slice(-25, lower_trim))

        cl_voltage = np.float_power(np.abs(fixed_plot_ds.voltage.values + 4), 0.75)
        
        if i == 0:
            all_plot_ds = combo_ds.sel(theta=theta, voltage=slice(-25,-4))
            ax[0].plot(np.float_power(np.abs(all_plot_ds.voltage.values + 4), 0.75), 
                       all_plot_ds.current_i.values, label=theta)
            ax[0].set_ylabel(r'$I_i$')
            ax[0].set_xlabel(r'$|V|^{\frac{3}{4}}$')
            ax[0].legend()
        
        fit_data = se_fitter.fit(cl_voltage, fixed_plot_ds.current_i.values, sigma=fixed_plot_ds.derr_current_i.values)
        
        thetas.append(theta)
        seps.append(fit_data.get_sheath_exp())
        d_seps.append(fit_data.get_sheath_exp_err())
        
    combo_ds['ion_a'] = (combo_ds['theta'] * 0) + np.array(seps)

    plot_decomp(combo_ds, sheath_label='ion_a', mode=1, ax=ax[1], colour=cb_palette[i], plot_label=lower_trim)
ax[1].legend(bbox_to_anchor=(1.0, 1.25), loc='upper center', borderaxespad=0.)
fig.tight_layout()

## Compare between fixed-wall-potential, piecewise concatenation and floating potential sims
All at $\theta = 10^{\circ}$

In [63]:
fwp_dir_path = list((spice_dir / 'marconi/spice2/sheath_exp_fwp/flat_10').glob('*'))[0]
print(fwp_dir_path)
fwp_backup_path = list(fwp_dir_path.glob('backup*'))[0]


/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp_fwp/flat_10/alpha_yz_-10.0


In [64]:
splopter =  spl.Splopter(fwp_backup_path, reduce=desired_variables, ignore_tzero_fl=True, version=2.14,
                         store_dataframe_fl=True, )

splopter.prepare(find_se_temp_fl=False, homogenise_fl=True, backup_concat_fl=False)

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp_fwp/flat_10/alpha_yz_-10.0/backup_20201117-1742) doesn't seem to be valid.
Continuing anyway.


/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/scipy/io/matlab/mio.py:218: MatReadWarning: Duplicate variable name "npartproc" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


In [65]:
splopter.iv_data.plot()

<ErrorbarContainer object of 3 artists>

In [66]:
new_iv_data = splopter.iv_data.get_below_floating()

In [69]:
fig, ax = plt.subplots()

sweep_plot_ds = swept_ds.sel(theta_p=0.0, recession=0.0, theta=10, voltage=slice(-10,0))
pw_plot_ds = combo_ds.sel(theta=-10.0, voltage=slice(-10,0))

splopter.iv_data.plot(ax=ax)
sweep_plot_ds['current'].plot.line(x='voltage', ax=ax, label='')
(-pw_plot_ds['current']).plot.line(x='voltage', ax=ax)

In [71]:
se_fitter = fts.IonCurrentSEFitter()
v_f = splopter.iv_data.get_vf()
sweep_plot_ds = swept_ds.sel(theta_p=0.0, recession=0.0, theta=10, voltage=slice(-10, v_f))
pw_plot_ds = combo_ds.sel(theta=-10.0, voltage=slice(-25, v_f))

fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize=[12,6])
ax[0].plot(np.float_power(np.abs(new_iv_data['V'] - v_f), 0.75), new_iv_data['I_i'])
ax[0].set_title('Fixed wall potential')

fit_data = se_fitter.fit(np.float_power(np.abs(new_iv_data['V'] - v_f), 0.75), new_iv_data['I_i'])
fit_data.plot(ax=ax[0])

ax[1].plot(np.float_power(np.abs(sweep_plot_ds['voltage'] - v_f), 0.75), sweep_plot_ds['current_i'])
fit_data = se_fitter.fit(np.float_power(np.abs(sweep_plot_ds['voltage'] - v_f), 0.75), sweep_plot_ds['current_i'])
fit_data.plot(ax=ax[1])
ax[1].set_title('Floating wall potential')

ax[0].plot(np.float_power(np.abs(pw_plot_ds['voltage'] - v_f), 0.75), pw_plot_ds['current_i'])
fit_data = se_fitter.fit(np.float_power(np.abs(pw_plot_ds['voltage'] - v_f), 0.75), pw_plot_ds['current_i'])
fit_data.plot(ax=ax[0])
ax[2].set_title('Fixed wall potential - Piecewise')


fig.tight_layout()

### Stuff nicked from analysis_1.ipynb

In [52]:
parser = splopter.parser

dV = parser.getfloat('mks', 'mks_te') #* c.BOLTZMANN / c.ELEM_CHARGE
T_e = parser.getfloat('mks', 'mks_te')
n_0 = parser.getfloat('mks', 'mks_n0')

L = parser.getfloat('rectangle2', 'yhigh') - parser.getfloat('rectangle2', 'ylow')
g = parser.getfloat('rectangle2', 'ylow') - parser.getfloat('rectangle0', 'yhigh') 
lambda_D = lpu.debye_length(T_e, n_0)
theta_p = np.radians(10.0)

print(f"L = {L} L_d \n"
      f"g = {g} L_d \n"
      f"lambda_D = {lambda_D} \n"
      f"theta_p = {theta_p} \n")

L = 300.0 L_d 
g = 60.0 L_d 
lambda_D = 1.6622799720325184e-05 
theta_p = 0.17453292519943295 



In [53]:
print(2.38e-3 / lpu.debye_length(6, 6.4e18))
print(5e-3 / lpu.debye_length(6, 6.4e18))

330.6527573670192
694.6486499307126


In [54]:
def decompose_new_sheath_exp_param(a, theta, L, g, d_perp, theta_p):
    y = (a * np.sqrt(np.sin(theta)) * (((L + g) * np.tan(theta)) 
                                       + (L * np.tan(theta_p)) - d_perp))
    x = np.tan(theta) + (2 * np.tan(theta_p))
    return x, y

def decompose_alt_new_sheath_exp_param(a, theta, L, g, d_perp, theta_p):
    y = (a * np.sqrt(np.sin(theta)) * (L + g + (((L * np.tan(theta_p)) - d_perp) / np.tan(theta))))
    x = np.cos(theta) / np.sin(theta)
    return x, y

def decompose_2d_box_sheath_exp_param(a, theta, L, g, d_perp, theta_p):
    L_eff = (L/np.cos(theta_p)) - ((d_perp - (g * np.tan(theta))) 
                                   / ((np.cos(theta_p) * np.tan(theta)) + np.sin(theta_p)))
    
    y = a * np.sqrt(np.sin(theta + theta_p)) * L_eff
    x = np.cos(theta + theta_p) / np.sin(theta + theta_p)
    return x, y

def decompose_sheath_exp_param(a, theta, L, g, d_perp=0, theta_p=0):
    y = a * (L + g) * np.sqrt(np.sin(theta))
    x = np.cos(theta) / np.sin(theta)
    return x, y


In [55]:
decomp_modes = [
    None, 
    decompose_sheath_exp_param, 
    decompose_new_sheath_exp_param, 
    decompose_2d_box_sheath_exp_param,
    decompose_alt_new_sheath_exp_param
]
mode_labels = [
    None, 
    (r'$\cot{\theta}$', r'$a\sin^{1/2}{\theta}\cdot[L + g]$'),
    (r'$\tan{\theta} + 2\tan{\theta_p}$', r'$a\sin^{1/2}{\theta}[(L+g)\tan{\theta} + L\tan{\theta_p} - d_{\perp}]$'),
    (r'$\cot{\theta_{tot}}$', r'$a\sin^{1/2}{\theta_{tot}}[L_{eff} + \Delta_0 \cot{\theta_{tot}}]$'),
    (r'$\cot{\theta}$', r'$a\sin^{1/2}{\theta}[L + g + (L\tan{\theta_p} - d_{\perp}) * \cot{\theta}]$'),
]
mode_constants = [
    None,
    ('y_0', 'm'),
    ('m', 'y_0'),
    ('y_0', 'm'),
    ('y_0', 'm'),
]

def plot_decomp(ds, sheath_label='ion_a', mode=1, fit_fl=True, ax=None, kwargs_for_plot={}, 
                kwargs_for_fitplot={}, colour='r', plot_label=None):
    if ax is None:
        fig, ax = plt.subplots()
    else:
        fig = ax.figure
    
    if mode in [1, 2, 3, 4]:
        x, y = decomp_modes[mode](ds[sheath_label], ds['theta_rads'], L, g, 
                                  ds['recession'], ds['theta_p_rads'])
    else:
        raise ValueError('Mode must be 1, 2 or 3')
    
    x = x[~np.isnan(y)]
    y = y[~np.isnan(y)]
    
    if plot_label is None:
        plot_label = sheath_label
    
    ax.errorbar(x, y, yerr=ds[sheath_label]*y, color=colour, label=plot_label, **kwargs_for_plot)
    
    sl_fitter = fts.StraightLineFitter()
    fit_data = sl_fitter.fit(x, y)
    if fit_fl:    
        c1, c2 = mode_constants[mode]
        fit_label = r'$c_1$ = {:.2g}, $c_2$ = {:.2g}'.format(fit_data.get_param(c1), fit_data.get_param(c2))
        ax.plot(*fit_data.get_fit_plottables(), color=colour, label=fit_label, 
                **kwargs_for_fitplot)
        
    ax.set_xlabel(mode_labels[mode][0])
    ax.set_ylabel(mode_labels[mode][1])
#     ax.legend()
    
    return ax, fit_data